# install & setup

In [ ]:
!pip install opencv-python ultralytics numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls

# Model

In [ ]:
from ultralytics import YOLO
import random
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

model = YOLO("yolo11x-seg.pt")

In [ ]:
imgfile = '/content/drive/MyDrive/Colab Notebooks/datasets/DL/sample_city.jpg'
# imgfile = '/content/drive/MyDrive/Colab Notebooks/datasets/DL/cat615m.jpg'
# imgfile = '/content/drive/MyDrive/Colab Notebooks/datasets/DL/bird1.jpg'
# img = cv2.imread(imgfile)
# img

try:
  img = cv2.imread(imgfile)
  if img is not None:
    cv2_imshow(img)
  else:
    print(f"Error: Could not read image {imgfile} ")
except Exception as e:
  print(f"An error occurred: {e}")

In [ ]:
# imgfile = '/content/drive/MyDrive/Colab Notebooks/datasets/DL/sample_city.jpg'
# imgfile = '/content/drive/MyDrive/Colab Notebooks/datasets/DL/cat615m.jpg'
# img = cv2.imread(imgfile)

# all classes
yolo_classes = list(model.names.values())
classes_ids = [yolo_classes.index(clas) for clas in yolo_classes]

conf = 0.2

results = model.predict(img, conf=conf)
colors = [random.choices(range(256), k=3) for _ in classes_ids]

for result in results:
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        color_number = classes_ids.index(int(box.cls[0]))
        cv2.fillPoly(img, points, colors[color_number])

# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.imwrite("YourSavePath", img)

In [ ]:
img

# Mask and crop

In [ ]:
# create mask for cropping

import numpy as np

def create_mask(results, img_shape):
  """Creates a mask for cropping based on YOLO segmentation results.

  Args:
      results: YOLO prediction results.
      img_shape: Shape of the original image (height, width).

  Returns:
      A binary mask (NumPy array) where 1 represents the area to keep.
  """

  mask = np.zeros(img_shape[:2], dtype=np.uint8)
  for result in results:
      for box, mask_data in zip(result.boxes, result.masks.xy):
          # keep the areas with detected objects
          points = np.int32([mask_data])
          cv2.fillPoly(mask, points, 255)  # Fill the polygon with white (255)
  return mask


#  'results' from your YOLO prediction)
img = cv2.imread(imgfile)
img_shape = img.shape  # Get shape of the image
combined_mask = create_mask(results, img_shape)

# # To crop the image based on the mask:
cropped_img = cv2.bitwise_and(img, img, mask=combined_mask)

cv2_imshow(cropped_img)


In [ ]:
#  white background

# Create a white background image
white_background = np.full(img.shape, 255, dtype=np.uint8)

# Use the combined mask to extract the object from the original image
extracted_object = cv2.bitwise_and(img, img, mask=combined_mask)

# Combine the extracted object with the white background
final_image = np.where(combined_mask[..., None] == 0, white_background, extracted_object)


cv2_imshow(final_image)

In [ ]:
combined_mask

In [ ]:
for r in results:
    print(r.boxes)
# yolo_classes